# Amazon SageMaker の PyTorch 推論機能をまとめてご紹介

> *このノートブックは、Amazon SageMaker Studio の `Python 3 (PyTorch 1.6 Python 3.6 CPU Optimized)` カーネルや、SageMaker ノートブックインスタンスの `conda_pytorch_p36` カーネルでご利用ください。*

このサンプルノートブックでは、Amazon SageMaker の主に推論関連の機能を PyTorch を使って使用する方法をご紹介します。

以下の手順 1, 2 の[必須]の部分を実行したあとは、マルチモデルエンドポイント、非同期推論、カスタムコンテナイメージなどのどこから実施いただいても構いません。

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#[必須]-ライブラリや-SDK-の準備" data-toc-modified-id="[必須]-ライブラリや-SDK-の準備-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>[必須] ライブラリや SDK の準備</a></span></li><li><span><a href="#[必須]-サンプルデータセット:-MNIST" data-toc-modified-id="[必須]-サンプルデータセット:-MNIST-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>[必須] サンプルデータセット: MNIST</a></span></li><li><span><a href="#SageMaker-SDK-を使ってマルチモデルエンドポイントを作成" data-toc-modified-id="SageMaker-SDK-を使ってマルチモデルエンドポイントを作成-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>SageMaker SDK を使ってマルチモデルエンドポイントを作成</a></span><ul class="toc-item"><li><span><a href="#ローカルモードでモデルを学習" data-toc-modified-id="ローカルモードでモデルを学習-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>ローカルモードでモデルを学習</a></span></li><li><span><a href="#複数のモデルを学習" data-toc-modified-id="複数のモデルを学習-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>複数のモデルを学習</a></span></li><li><span><a href="#単独でのモデルのデプロイ" data-toc-modified-id="単独でのモデルのデプロイ-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>単独でのモデルのデプロイ</a></span></li><li><span><a href="#Estimator-から-SageMaker-Model-を作成" data-toc-modified-id="Estimator-から-SageMaker-Model-を作成-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Estimator から SageMaker Model を作成</a></span></li><li><span><a href="#Amazon-SaegMaker-MultiDataModel-エンティティを作成" data-toc-modified-id="Amazon-SaegMaker-MultiDataModel-エンティティを作成-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Amazon SaegMaker MultiDataModel エンティティを作成</a></span></li><li><span><a href="#マルチモデルエンドポイントをデプロイ" data-toc-modified-id="マルチモデルエンドポイントをデプロイ-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>マルチモデルエンドポイントをデプロイ</a></span></li><li><span><a href="#エンドポイントで利用可能なモデルを確認" data-toc-modified-id="エンドポイントで利用可能なモデルを確認-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>エンドポイントで利用可能なモデルを確認</a></span></li><li><span><a href="#動的にモデルをエンドポイントにデプロイ" data-toc-modified-id="動的にモデルをエンドポイントにデプロイ-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>動的にモデルをエンドポイントにデプロイ</a></span></li><li><span><a href="#エンドポイントで利用可能なモデルを確認" data-toc-modified-id="エンドポイントで利用可能なモデルを確認-3.9"><span class="toc-item-num">3.9&nbsp;&nbsp;</span>エンドポイントで利用可能なモデルを確認</a></span></li><li><span><a href="#マルチモデルエンドポイントで推論の実行" data-toc-modified-id="マルチモデルエンドポイントで推論の実行-3.10"><span class="toc-item-num">3.10&nbsp;&nbsp;</span>マルチモデルエンドポイントで推論の実行</a></span></li><li><span><a href="#エンドポイントの削除" data-toc-modified-id="エンドポイントの削除-3.11"><span class="toc-item-num">3.11&nbsp;&nbsp;</span>エンドポイントの削除</a></span></li><li><span><a href="#マルチモデルエンドポイントのモデルの追加" data-toc-modified-id="マルチモデルエンドポイントのモデルの追加-3.12"><span class="toc-item-num">3.12&nbsp;&nbsp;</span>マルチモデルエンドポイントのモデルの追加</a></span></li></ul></li><li><span><a href="#マルチモデルエンドポイントのトラフィック分散" data-toc-modified-id="マルチモデルエンドポイントのトラフィック分散-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>マルチモデルエンドポイントのトラフィック分散</a></span><ul class="toc-item"><li><span><a href="#2つのモデルを学習" data-toc-modified-id="2つのモデルを学習-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>2つのモデルを学習</a></span></li><li><span><a href="#エンドポイントの削除" data-toc-modified-id="エンドポイントの削除-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>エンドポイントの削除</a></span></li></ul></li><li><span><a href="#マルチコンテナエンドポイント" data-toc-modified-id="マルチコンテナエンドポイント-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>マルチコンテナエンドポイント</a></span><ul class="toc-item"><li><span><a href="#デプロイするモデルの学習" data-toc-modified-id="デプロイするモデルの学習-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>デプロイするモデルの学習</a></span></li><li><span><a href="#コンテナ呼び出し情報の設定" data-toc-modified-id="コンテナ呼び出し情報の設定-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>コンテナ呼び出し情報の設定</a></span></li><li><span><a href="#マルチコンテナエンドポイントの作成" data-toc-modified-id="マルチコンテナエンドポイントの作成-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>マルチコンテナエンドポイントの作成</a></span></li><li><span><a href="#推論の実行" data-toc-modified-id="推論の実行-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>推論の実行</a></span></li></ul></li><li><span><a href="#非同期推論" data-toc-modified-id="非同期推論-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>非同期推論</a></span><ul class="toc-item"><li><span><a href="#デプロイするモデルの学習" data-toc-modified-id="デプロイするモデルの学習-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>デプロイするモデルの学習</a></span></li><li><span><a href="#コンテナイメージの取得" data-toc-modified-id="コンテナイメージの取得-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>コンテナイメージの取得</a></span></li><li><span><a href="#モデルの作成" data-toc-modified-id="モデルの作成-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>モデルの作成</a></span></li><li><span><a href="#エンドポイント設定の作成" data-toc-modified-id="エンドポイント設定の作成-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>エンドポイント設定の作成</a></span></li><li><span><a href="#エンドポイントの作成" data-toc-modified-id="エンドポイントの作成-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>エンドポイントの作成</a></span></li><li><span><a href="#[オプション]-オートスケールポリシーの設定" data-toc-modified-id="[オプション]-オートスケールポリシーの設定-6.6"><span class="toc-item-num">6.6&nbsp;&nbsp;</span>[オプション] オートスケールポリシーの設定</a></span></li><li><span><a href="#非同期推論エンドポイントで推論" data-toc-modified-id="非同期推論エンドポイントで推論-6.7"><span class="toc-item-num">6.7&nbsp;&nbsp;</span>非同期推論エンドポイントで推論</a></span></li><li><span><a href="#リソースの削除" data-toc-modified-id="リソースの削除-6.8"><span class="toc-item-num">6.8&nbsp;&nbsp;</span>リソースの削除</a></span></li></ul></li><li><span><a href="#カスタムコンテナイメージで学習・推論" data-toc-modified-id="カスタムコンテナイメージで学習・推論-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>カスタムコンテナイメージで学習・推論</a></span><ul class="toc-item"><li><span><a href="#学習用コンテナイメージの作成" data-toc-modified-id="学習用コンテナイメージの作成-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>学習用コンテナイメージの作成</a></span></li><li><span><a href="#推論用コンテナイメージの作成" data-toc-modified-id="推論用コンテナイメージの作成-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>推論用コンテナイメージの作成</a></span></li><li><span><a href="#カスタムコンテナイメージを使って推論" data-toc-modified-id="カスタムコンテナイメージを使って推論-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>カスタムコンテナイメージを使って推論</a></span></li></ul></li><li><span><a href="#Inf1-インスタンスで推論" data-toc-modified-id="Inf1-インスタンスで推論-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Inf1 インスタンスで推論</a></span><ul class="toc-item"><li><span><a href="#モデルの学習" data-toc-modified-id="モデルの学習-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>モデルの学習</a></span></li><li><span><a href="#モデルのコンパイル" data-toc-modified-id="モデルのコンパイル-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>モデルのコンパイル</a></span></li><li><span><a href="#Inf1-インスタンスで推論エンドポイントを作成し推論" data-toc-modified-id="Inf1-インスタンスで推論エンドポイントを作成し推論-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Inf1 インスタンスで推論エンドポイントを作成し推論</a></span></li></ul></li><li><span><a href="#サーバレス推論-(preview)" data-toc-modified-id="サーバレス推論-(preview)-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>サーバレス推論 (preview)</a></span><ul class="toc-item"><li><span><a href="#デプロイするモデルの学習" data-toc-modified-id="デプロイするモデルの学習-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>デプロイするモデルの学習</a></span></li><li><span><a href="#サーバレスエンドポイントの作成" data-toc-modified-id="サーバレスエンドポイントの作成-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>サーバレスエンドポイントの作成</a></span></li><li><span><a href="#サーバレス推論の実行" data-toc-modified-id="サーバレス推論の実行-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>サーバレス推論の実行</a></span></li></ul></li><li><span><a href="#リソースの削除" data-toc-modified-id="リソースの削除-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>リソースの削除</a></span></li></ul></div>

## [必須] ライブラリや SDK の準備

以下のセルを実行したあと、一度このノートブックのカーネルを再起動してください。再起動は、メニューの「Kernel」->「Restart」で実行できます。再起動後、以下のセルを再度実行する必要はありません。

以下のエラーが表示されることがありますが、ノートブックの実行には問題ありません。
```
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 1.3.0 requires botocore<1.20.50,>=1.20.49, but you have botocore 1.21.37 which is incompatible.
```

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip --quiet
!{sys.executable} -m pip install -U awscli sagemaker boto3 --quiet

In [ ]:
import sagemaker
sagemaker.__version__

In [ ]:
# Python Built-Ins:
from datetime import datetime
import os
import json
import logging
from tempfile import TemporaryFile
import time

# External Dependencies:
import boto3
from botocore.exceptions import ClientError
import numpy as np
import sagemaker
from sagemaker.multidatamodel import MultiDataModel
from sagemaker.pytorch import PyTorch

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

boto_session = boto3.session.Session()
sagemaker_client = boto_session.client("sagemaker")
sagemaker_runtime = boto_session.client("sagemaker-runtime")
region = boto_session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

# Configuration:
bucket_name = sagemaker_session.default_bucket()
prefix = "mnist"
output_path = f"s3://{bucket_name}/{prefix}"

In [ ]:
import os, json
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get("DomainId")
        on_studio = True if domain_id is not None else False
print("Is this notebook runnning on Studio?: {}".format(on_studio))

## [必須] サンプルデータセット: MNIST

MNIST は、手書き数字の分類に広く用いられているデータセットです。このデータセットは、7万枚のラベル付き 28x28画素の手書き数字のグレースケール画像から構成されています。このデータセットは、6万枚の学習用画像と 1万枚のテスト用画像に分かれています。

このサンプルノートブックでは、MNISTデータをパブリックの S3バケットからダウンロードし、`bucket_name` に設定したデフォルトの SageMakerバケットにアップロードします。

In [ ]:
!aws s3 cp s3://fast-ai-imageclas/mnist_png.tgz . --no-sign-request
if on_studio:
    !tar -xzf mnist_png.tgz -C /opt/ml --no-same-owner
else:
    !tar -xvzf  mnist_png.tgz

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os

root_dir_studio = '/opt/ml'
data_dir = os.path.join(root_dir_studio,'data') if on_studio else 'data'
training_dir = os.path.join(root_dir_studio,'mnist_png/training') if on_studio else 'mnist_png/training'
test_dir = os.path.join(root_dir_studio,'mnist_png/testing') if on_studio else 'mnist_png/testing'

os.makedirs(data_dir, exist_ok=True)

training_data = datasets.ImageFolder(root=training_dir,
                            transform=transforms.Compose([
                            transforms.Grayscale(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))
test_data = datasets.ImageFolder(root=test_dir,
                            transform=transforms.Compose([
                            transforms.Grayscale(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))

training_data_loader = DataLoader(training_data, batch_size=len(training_data))
training_data_loaded = next(iter(training_data_loader))
torch.save(training_data_loaded, os.path.join(data_dir, 'training.pt'))

test_data_loader = DataLoader(test_data, batch_size=len(test_data))
test_data_loaded = next(iter(test_data_loader))
torch.save(test_data_loaded, os.path.join(data_dir, 'test.pt'))

training.pt, test.pt として保存した学習用、テスト用データを以下のコマンドで S3 にアップロードします。

In [ ]:
inputs = sagemaker_session.upload_data(path=data_dir, bucket=bucket_name, key_prefix=os.path.join(prefix, 'data'))
print('input spec (in this case, just an S3 path): {}'.format(inputs))

推論テスト用に、テスト用データセットからランダムに5つデータを取得します。

In [ ]:
%matplotlib inline
import random
import numpy as np
import matplotlib.pyplot as plt

raw_test_data = datasets.ImageFolder(root=test_dir,
                                        transform=transforms.Compose([
                                        transforms.Grayscale(),
                                        transforms.ToTensor()]))
num_samples = 5
indices = random.sample(range(len(raw_test_data) - 1), num_samples)
raw_images = np.array([raw_test_data[i][0].numpy() for i in indices])
raw_labels = np.array([raw_test_data[i][1] for i in indices])


for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(raw_images[i].reshape(28, 28), cmap='gray')
    plt.title(raw_labels[i])
    plt.axis('off')
    
images = np.array([test_data[i][0].numpy() for i in indices])

## SageMaker SDK を使ってマルチモデルエンドポイントを作成

Amazon SageMaker の [マルチモデルエンドポイント](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html) では、最大数千のモデルをシームレスにホストするエンドポイントを作成することができます。これらのエンドポイントは、共通の推論コンテナから提供される多数のモデルのいずれかがオンデマンドで呼び出される必要があり、頻繁に呼び出されないモデルで推論する際に多少のレイテンシーが生じても問題ないユースケースに適しています。低い推論レイテンシーが常時必要とされるアプリケーションには、従来のエンドポイントが最適です。

変動するレイテンシーが許容範囲内であり、コストの最適化がより重要な場合、マルチモデルエンドポイントのしくみを使って [A/B/n テスト](https://aws.amazon.com/jp/blogs/news/a-b-testing-ml-models-in-production-using-amazon-sagemaker/) を行うお客様もいらっしゃいます。

このノートブックでは、マルチモデルエンドポイントの作成および使用方法を示すために、SageMaker の [PyTorchフレームワークコンテナ](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html) で学習されたモデルを使用した例をご紹介します。ここでは簡単に A/B シナリオを取り上げ、2つのモデルを学習してエンドポイントにデプロイします。

PyTorch で マルチモデルエンドポイントを作成するには、学習スクリプトに 2つの関数を含めておく必要があります。これは、推論時に使用するソースコードなどを model.tar.gz に含めるためです。このサンプルでは、`code/train.py` にマルチモデルエンドポイント用の関数 `enable_sm_oneclick_deploy()`, `enable_torchserve_multi_model()` が定義され、それらが `save_model()` の中でで呼び出されています。

マルチモデルエンドポイントの仕組みについては [こちらのドキュメント](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/multi-model-endpoints.html) をご参照ください。

カスタムコンテナイメージを使ってマルチモデルエンドポイントを作成する場合は、[こちらのサンプルコード](https://github.com/aws/amazon-sagemaker-examples/tree/master/advanced_functionality/multi_model_bring_your_own) や [こちらのドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/build-multi-model-build-container.html) をご参照ください。



### ローカルモードでモデルを学習

学習スクリプトを変更したときや、コンテナイメージを更新したとき、問題なく動作するかどうかを簡単に確認したいことがあります。ローカルモードを使うことで、新しいインスタンスを起動するのではなく、ノートブックインスタンス上で（ローカルPC で `estimator.fit()` を実行する場合はローカルPC上で）学習ジョブを立ち上げることができるため、インスタンス起動を待つことなくすぐに動作確認をすることができます。方法は、Estimator を作成する際の引数 `instance_type` に `local` と設定するだけです。**なお、SageMaker Studio ではローカルモードを使用することはできません。**

In [ ]:
if on_studio:
    instance_type = 'ml.m4.xlarge'
else:
    instance_type = 'local'

estimator = PyTorch(entry_point="train.py",
                    source_dir='code_mme',
                    role=role,
                    framework_version='1.8.0',
                    py_version='py3',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters={
                        'batch-size':128,
                        'lr': 0.01,
                        'epochs': 1,
                        'backend': 'gloo'
                    })

In [ ]:
estimator.fit({'training': inputs})

ローカルモードで、ノートブックインスタンス上に推論エンドポイントを作成します。

モデルの学習に使用したスクリプト（このサンプルノートブックの場合 `train.py`）に推論用のコード（`model_fn()`: 必須、`input_fn()`: 任意）が記載されていれば、`estimatorA.deploy()` で推論エンドポイントをデプロイすることも可能です。今回は、推論用スクリプト `inference.py` を使用するため、`create_model` で作成した model を使って推論エンドポイントを起動します。

In [ ]:
model = estimator.create_model(role=role, source_dir="code_mme", entry_point="inference.py")
predictor = model.deploy(initial_instance_count=1, instance_type=instance_type)

上記セルの左側にある `In [*]` のアスタリスクが数字に変わったら、以下のセルを実行して推論します。

In [ ]:

prediction = predictor.predict(images)
predicted_label = prediction.argmax(axis=1)
print('The predicted labels are: {}'.format(predicted_label))

問題なく推論できることが確認できました。以下のコマンドで predictor を削除します。

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)

### 複数のモデルを学習

ここからは、同じデータセットを使って 2つのモデルを学習させます。モデルの学習には SageMaker の PyTorch フレームワークコンテナを使用します。

シンプルな例として、同じソースコードを使って、ハイパーパラメタを少し変えてモデル A と B の2つを作成します。

In [ ]:
def get_estimator(base_job_name, hyperparam_overrides={}):
    hyperparameters = {
        "epochs": 20,
        "lr": 1e-3,
    }
    for k, v in hyperparam_overrides.items():
        hyperparameters[k] = v


    return PyTorch(
        base_job_name=base_job_name,
        entry_point="train.py",
        source_dir="code_mme",  # directory of your training script
        role=role,
        framework_version="1.8.0",
        py_version="py3",
        instance_type="ml.c4.xlarge",
        instance_count=1,
        output_path=output_path,
        hyperparameters=hyperparameters,
    )


estimatorA = get_estimator(base_job_name="mnist-a", hyperparam_overrides={"batch-size": 128})
estimatorB = get_estimator(base_job_name="mnist-b", hyperparam_overrides={"batch-size": 64})

デフォルトでは、 [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/) の [Estimator.fit()](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit) メソッドを呼び出すと、トレーニングジョブが完了するまで待機状態となり、進捗情報とログをノートブックにストリーミングします。

しかし、サポートされている設定はこれだけではありません。例えば、`wait=False` を設定して非同期にジョブを開始したり、過去に開始されたジョブを遡って `wait()` することも可能です（オプションでログを取得することもできます）。

以下のセクションでは、2つのトレーニングジョブを並行して起動し、Bの実行時にログをストリームし、Aが完了していなければAの完了を待ちます。

In [ ]:
estimatorA.fit({"training": inputs}, wait=False)
print("Started estimator A training in background (logs will not show)")

print("Training estimator B with logs:")
estimatorB.fit({"training": inputs})

print("\nWaiting for estimator A to complete:")
estimatorA.latest_training_job.wait(logs=False)

### 単独でのモデルのデプロイ

モデルA を推論エンドポイントにデプロイしてみます。エンドポイントの起動が完了するまでに 10分ほどかかることがあります。

モデルの学習に使用したスクリプト（このサンプルノートブックの場合 `train.py`）に推論用のコード（`model_fn()`: 必須、`input_fn()`: 任意）が記載されていれば、`estimatorA.deploy(...)` で推論エンドポイントをデプロイすることも可能です。今回は、推論用スクリプト `inference.py` を使用するため、以下のコードで推論エンドポイントを起動します。

In [ ]:
modelA = estimatorA.create_model(role=role, source_dir="code_mme", entry_point="inference.py")

In [ ]:
predictorA = modelA.deploy(
    initial_instance_count=1,
    instance_type="ml.c5.xlarge",
)

In [ ]:
start_time = time.time()
prediction = predictorA.predict(images)
duration = time.time() - start_time

print(f"Model took {int(duration * 1000/num_samples):,d} ms")

predicted_label = prediction.argmax(axis=1)

print('The GT labels are: {}'.format(raw_labels))
print('The predicted labels are: {}'.format(predicted_label))

すでに起動済みのエンドポイントを使って推論する場合は、エンドポイント名を使って以下の手順で predictor を作成することができます。

In [ ]:
from sagemaker.pytorch.model import PyTorchPredictor

predictorA2 = PyTorchPredictor(
        endpoint_name=predictorA.endpoint_name
)


In [ ]:

prediction = predictorA.predict(images)
predicted_labelA = prediction.argmax(axis=1)

prediction = predictorA2.predict(images)
predicted_labelA2 = prediction.argmax(axis=1)

print('The GT labels are: {}'.format(raw_labels))
print('The predicted labels (A) are: {}'.format(predicted_labelA))
print('The predicted labels (A2) are: {}'.format(predicted_labelA2))


作成した推論エンドポイントは、起動している間料金が発生するので、不要になったら速やかに削除しましょう。以下のコマンドを実行するか、SageMaker のコンソールのエンドポイント一覧画面から削除してください。

In [ ]:
predictorA.delete_endpoint(delete_endpoint_config=True)

### Estimator から SageMaker Model を作成

マルチモデルエンドポイントは、共通のコンテナの中にあるモデルをロードします。そのため、まずいずれかの Estimator を使って Model を作成します。

In [ ]:
model = estimatorA.create_model(role=role, source_dir="code_mme", entry_point="inference.py")

### Amazon SaegMaker MultiDataModel エンティティを作成

 [MultiDataModel](https://sagemaker.readthedocs.io/en/stable/api/inference/multi_data_model.html) クラスを使用して、マルチモデルのエンドポイントを作成します。

`sagemaker.model.Model` オブジェクトを直接渡して MultiDataModel を作成することができます。この場合、MultiDataModel がデプロイされると、エンドポイントは使用するイメージや環境変数、ネットワークの分離などの情報を引継ぎます。

また、`sagemaker.model.Model` オブジェクトを明示的に渡さずに MultiDataModel を作成することも可能です。詳細については [ドキュメント](https://sagemaker.readthedocs.io/en/stable/api/inference/multi_data_model.html) をご参照ください。

以下のセルを実行して、マルチモデルエンドポイントが参照するモデルを格納するためのパスを作成します。

In [ ]:
# This is where our MME will read models from on S3.
multi_model_prefix = f"{prefix}/multi-model/"
multi_model_s3uri = f"s3://{bucket_name}/{multi_model_prefix}"
print(multi_model_s3uri)

In [ ]:
mme = MultiDataModel(
    name="mnist-multi-" + datetime.now().strftime("%Y-%m-%d-%H-%M-%S"),
    model_data_prefix=multi_model_s3uri,
    model=model,  # passing our model
    sagemaker_session=sagemaker_session,
)

### マルチモデルエンドポイントをデプロイ

マルチモデルエンドポイントで使用する予定のすべてのモデルの予測ワークロードに対して、適切なインスタンスタイプとインスタンス数を検討する必要があります。また、デプロイするモデルの数やサイズによって、必要なメモリ容量も変わってきます。

In [ ]:
try:
    predictor.delete_endpoint(delete_endpoint_config=True)
    print("Deleting previous endpoint...")
    time.sleep(10)
except (NameError, ClientError, FileNotFoundError):
    pass

predictor = mme.deploy(
    initial_instance_count=1,
    instance_type="ml.c5.xlarge",
)


既存のマルチモデルエンドポイントについても、単一モデルエンドポイントと同様に以下のコードで `predictor` を作成することができます。

In [ ]:
# from sagemaker.pytorch.model import PyTorchPredictor

# mme_predictor = PyTorchPredictor(
#         endpoint_name='<endpoint name>'
# )

### エンドポイントで利用可能なモデルを確認

利用可能とは、`MultiDataModel` をセットアップする際に定義した S3 prefix （`model_data_prefix`） の下に、現在どのようなモデルアーティファクトが保存されているかということです。

現在は、定義した S3 prefix の下にアーティファクト （例：`tar.gz` ファイル） が保存されていないため、エンドポイントでは推論リクエストに対応できるモデルが「利用できない状態になっています。

以下では、エンドポイントでモデルを「利用可能」にする方法を説明します。

In [ ]:
# No models visible!
list(mme.list_models())

### 動的にモデルをエンドポイントにデプロイ

`MultiDataModel` の `.add_model()` メソッドは、トレーニングジョブが保存したモデルアーティファクトを、エンドポイントが参照する場所にコピーします。

以下のように、このメソッドを使い続けることで、必要に応じて稼働中のエンドポイントに動的にモデルをデプロイすることができます。

`model_data_source` は、モデルアーティファクトの場所を指します（例：学習完了後に S3にアップロードされた場所）。

`model_data_path` は、上記で指定した S3 prefix （例：`model_data_prefix`） への**相対パス**で、エンドポイントが推論に使用するモデルを特定するのに使用します。これは**相対パス**なので、モデルアーティファクト名のみを渡します。

> **注**： ここで紹介しているように、トレーニングジョブの `model.tar.gz` の出力を直接利用するためには、トレーニングジョブが以下のような結果を出すようにする必要があります。
>
> 推論に必要なファイルが `code/` サブフォルダに保存されている
> `code/` が(SageMaker PyTorch containers v1.6+を使用している場合)TorchServeと互換性がある形でパッケージ化されている
> 
> この点については、`code/train.py` の `enable_sm_oneclick_deploy()` 関数と `enable_torchserve_multi_model()` 関数を参照してください。また、同じ手順を事後に行って、トレーニングジョブがアップロードした `model.tar.gz` から、マルチモデルエンドポイントに対応した新しい `model.tar.gz` を作成することもできます。

In [ ]:
for name, est in {"ModelA": estimatorA, "ModelB": estimatorB}.items():
    artifact_path = est.latest_training_job.describe()["ModelArtifacts"]["S3ModelArtifacts"]
    # This is copying over the model artifact to the S3 location for the MME.
    mme.add_model(model_data_source=artifact_path, model_data_path=name)

### エンドポイントで利用可能なモデルを確認

`MultiDataModel` の設定時に指定した S3 prefix に、モデルのアーティファクトがリストアップされていることがわかります。これにより、エンドポイントはこれらのモデルに対する推論リクエストを投げられるようになりました。

In [ ]:
list(mme.list_models())

### マルチモデルエンドポイントで推論の実行

`mme.deploy()` を実行すると、`predictor` という変数に保存されている [RealTimePredictor](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/predictor.py#L35) が return されます。

単一モデルエンドポイントと同様に、その `predictor` を使って推論を実行することができますが、マルチモデルエンドポイントでは `target_model` を使ってどのモデルを呼び出すかを推論時に指定する必要があります。

以下のセルを実行することでマルチモデルエンドポイントを使った推論が実行されます。初めて以下のセルを実行したときと、2回目以降に実行したときで、推論にかかる時間はどのように変化したでしょうか？

In [ ]:
start_time = time.time()
prediction = predictor.predict(images, target_model="ModelB")
duration = time.time() - start_time

print(f"Model took {int(duration * 1000/num_samples):,d} ms per image")
predicted_label = prediction.argmax(axis=1)

print('The GT labels are: {}'.format(raw_labels))
print('The predicted labels are: {}'.format(predicted_label))

### エンドポイントの削除

エンドポイントは、 [SageMaker pricing page](https://aws.amazon.com/sagemaker/pricing/)　に記載の通り、稼働している時間に応じて課金されるため、使用しなくなったら削除する必要があります。ここでは、整理整頓のためにエンドポイントの設定も削除します。SageMaker のコンソールのエンドポイント一覧画面で、エンドポイントが削除されているかどうかを確認できます（一覧に表示されていなければ削除されたということです）。

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)

### マルチモデルエンドポイントのモデルの追加

モデルを更新するには、上記と同じ方法で、例えば `ModelA-2` のように新しいモデルとして追加します。

Amazon S3 に保存されたモデルアーティファクトを直接変更することは避けるべきです。なぜなら、古いバージョンのモデルがエンドポイントの実行中のコンテナや、エンドポイント上のインスタンスのストレージ・ボリュームにまだロードされている可能性があるからです。その場合、更新された S3 のモデルアーティファクトではなく、コンテナやストレージに保存されている古いバージョンのモデルが使用されることになります。

一度エンドポイントを停止して、新しいモデルを再デプロイすることが可能です。



## マルチモデルエンドポイントのトラフィック分散

ひとつのエンドポイントに複数のモデルをデプロイし、そのエンドポイントに推論リクエストを投げた際のトラフィックを指定した割合で分散させることができます。この機能を使って A/B テストを行うことができます。この機能を使う場合は Amazon SageMaker Python SDK ではなく AWS SDK for Python (boto3) を使用します。

### 2つのモデルを学習

以下のセルを実行して、エンドポイントにデプロイする 2つのモデルを学習します。**前のパートで既に estimatorA/B を学習済みの場合は以下のセルはスキップ可能です。**

In [ ]:
def get_estimator(base_job_name, hyperparam_overrides={}):
    hyperparameters = {
        "epochs": 20,
        "lr": 1e-3,
    }
    for k, v in hyperparam_overrides.items():
        hyperparameters[k] = v


    return PyTorch(
        base_job_name=base_job_name,
        entry_point="train.py",
        source_dir="code_mme",  # directory of your training script
        role=role,
        framework_version="1.8.0",
        py_version="py3",
        instance_type="ml.c4.xlarge",
        instance_count=1,
        output_path=output_path,
        hyperparameters=hyperparameters,
    )


estimatorA = get_estimator(base_job_name="mnist-a", hyperparam_overrides={"batch-size": 128})
estimatorB = get_estimator(base_job_name="mnist-b", hyperparam_overrides={"batch-size": 64})

estimatorA.fit({"training": inputs}, wait=False)
print("Started estimator A training in background (logs will not show)")

print("Training estimator B with logs:")
estimatorB.fit({"training": inputs})

print("\nWaiting for estimator A to complete:")
estimatorA.latest_training_job.wait(logs=False)

**estimatorA/B が学習済みの状態で以降のセルを実行してください。**

`create_model` を使って 2つのモデルを作成します。作成されたモデルは SageMaker コンソールの [推論] -> [モデル] のメニューから確認できます。`ModelDataUrl`　にデプロイしたいモデルが保存されている S3 パスを指定します。このサンプルでは estimator を使ってモデルの保存パスを取得していますが、デプロイしたいモデルが保存された S3 パスがわかっていれば直接 S3 パスを `ModelDataUrl` に指定すれば OK です。

In [ ]:
from sagemaker import image_uris

container_inference = image_uris.retrieve(
    region=region, framework="pytorch", 
    version="1.8.1", instance_type='ml.c5.xlarge', image_scope='inference', py_version='py36')

model_nameA = f"DEMO-pytorch-mnist-predA-{datetime.now():%Y-%m-%d-%H-%M-%S}"
model_nameB = f"DEMO-pytorch-mnist-predB-{datetime.now():%Y-%m-%d-%H-%M-%S}"

sagemaker_session.create_model(name=model_nameA, role=role, container_defs={
    'Image': container_inference,
    'ModelDataUrl': estimatorA.model_data
})

sagemaker_session.create_model(name=model_nameB, role=role, container_defs={
    'Image': container_inference,
    'ModelDataUrl': estimatorB.model_data
})

作成したモデルからそれぞれのモデルに対応する `production_variant` を作成します。

In [ ]:
from sagemaker.session import production_variant

variantA = production_variant(model_name=model_nameA,
                              instance_type="ml.m5.xlarge",
                              initial_instance_count=1,
                              variant_name='VariantA',
                              initial_weight=1)
                              
variantB = production_variant(model_name=model_nameB,
                              instance_type="ml.m5.xlarge",
                              initial_instance_count=1,
                              variant_name='VariantB',
                              initial_weight=1)

作成した `production_variant` を使ってエンドポイントをデプロイします。エンドポイントの起動状況は SageMaker コンソールの [推論] -> [エンドポイント] メニューから確認できます。

In [ ]:
endpoint_name = f"DEMO-pytorch-mnist-pred-{datetime.now():%Y-%m-%d-%H-%M-%S}"
print(f"EndpointName={endpoint_name}")

sagemaker_session.endpoint_from_production_variants(
    name=endpoint_name,
    production_variants=[variantA, variantB]
)


エンドポイントを使って推論します。以下のセルでは Amazon SageMaker Python SDK を使って推論しています。この方法では、推論結果の array のみが返ってきます。

In [ ]:
from sagemaker.pytorch.model import PyTorchPredictor

predictor = PyTorchPredictor(
        endpoint_name=endpoint_name
)

start_time = time.time()
prediction = predictor.predict(images)
duration = time.time() - start_time

print(f"Model took {int(duration * 1000/num_samples):,d} ms per image")
predicted_label = prediction.argmax(axis=1)

print('The GT labels are: {}'.format(raw_labels))
print('The predicted labels are: {}'.format(predicted_label))

print(prediction)

boto3 の `invoke_endpoint` を使って推論を実行すると、推論結果だけではなくどちらのモデルを使って推論が実行されたかなどの情報も併せて取得できます。

In [ ]:
import io
buffer= io.BytesIO()
np.save(buffer, images)

response = sagemaker_runtime.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType="application/x-npy",
                                   Body=buffer.getvalue())

response

推論結果は以下のコードで取得できます。

In [ ]:
import json
body = response['Body']
prediction = np.array(json.load(body))
predicted_label = prediction.argmax(axis=1)

print('The GT labels are: {}'.format(raw_labels))
print('The predicted labels are: {}'.format(predicted_label))

### エンドポイントの削除

エンドポイントは、 [SageMaker pricing page](https://aws.amazon.com/sagemaker/pricing/)　に記載の通り、稼働している時間に応じて課金されるため、使用しなくなったら削除する必要があります。ここでは、整理整頓のためにエンドポイントの設定も削除します。SageMaker のコンソールのエンドポイント一覧画面で、エンドポイントが削除されているかどうかを確認できます（一覧に表示されていなければ削除されたということです）。

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)

## マルチコンテナエンドポイント

上でご紹介したマルチモデルエンドポイントは、ひとつの推論エンドポイントにひとつのコンテナ、複数のモデルをデプロイするものです。デプロイしたいモデルがそれぞれ異なるコンテナイメージを必要とする場合、マルチコンテナエンドポイントをご利用できます。

このサンプルノートブックでは、SageMaker が用意したコンテナイメージを使っていますが、カスタムコンテナイメージを使ってマルチコンテナエンドポイントを作成することも可能です。カスタムコンテナイメージを使う方法は、このノートブックの「6  カスタムコンテナイメージで学習・推論」をご参照ください。

### デプロイするモデルの学習

PyTorch 1.8.0 と 1.8.1 のコンテナイメージを使ってそれぞれモデルを学習します。

In [ ]:
from sagemaker import image_uris

# Specify an AWS container image and region as desired
container_mce1 = image_uris.retrieve(
    region=region, framework="pytorch", 
    version="1.8.0", instance_type='ml.c5.xlarge', image_scope='training', py_version='py36')
container_mce1

In [ ]:
estimator_mce1 = PyTorch(entry_point="train.py",
                    image_uri=container_mce1,
                    source_dir='code_byoc',
                    role=role,
                    instance_count=1,
                    instance_type='ml.c5.xlarge',
#                     instance_type='local',
                    hyperparameters={
                        'batch-size':128,
                        'lr': 0.01,
                        'epochs': 10,
                        'backend': 'gloo'
                    })
estimator_mce1.fit({'training': inputs}, wait=False)

In [ ]:
from sagemaker import image_uris

# Specify an AWS container image and region as desired
container_mce2 = image_uris.retrieve(
    region=region, framework="pytorch", 
    version="1.8.1", instance_type='ml.c5.xlarge', image_scope='training', py_version='py36')
container_mce2

In [ ]:
estimator_mce2 = PyTorch(entry_point="train.py",
                    image_uri=container_mce2,
                    source_dir='code_byoc',
                    role=role,
                    instance_count=1,
                    instance_type='ml.c5.xlarge',
#                     instance_type='local',
                    hyperparameters={
                        'batch-size':128,
                        'lr': 0.01,
                        'epochs': 10,
                        'backend': 'gloo'
                    })
estimator_mce2.fit({'training': inputs})

以下のセルを実行して、推論用コンテナイメージの URI を取得します。

In [ ]:
from sagemaker import image_uris

# Specify an AWS container image and region as desired
container_pt180_mce1 = image_uris.retrieve(
    region=region, framework="pytorch", 
    version="1.8.0", instance_type='ml.c5.xlarge', image_scope='inference', py_version='py36')
print(container_pt180_mce1)

container_pt181_mce2 = image_uris.retrieve(
    region=region, framework="pytorch", 
    version="1.8.1", instance_type='ml.c5.xlarge', image_scope='inference', py_version='py36')
print(container_pt181_mce2)

### コンテナ呼び出し情報の設定

エンドポイントにデプロイしたコンテナを呼び出す際に必要な情報を辞書形式でまとめて設定します。

このノートブックでは、2つの　PyTorch モデルをデプロイしていますが、PyTorch モデルと Tensorflow モデルをデプロイするなども可能です。Tensorflow のモデルをデプロイする場合は、以下のセルのコメントアウト部分の書式にのっとってコンテナの設定を作成してください。

In [ ]:
# tensorflow_container = {
#     "ContainerHostname": "tensorflow-mnist",
#     "Image": tf_ecr_image_uri,
#     "ModelDataUrl": tf_mnist_model_data,
# }

pytorch_container1 = {
    "ContainerHostname": "pytorch-180-mnist",
    "Image": container_pt180_mce1,
    "ModelDataUrl": estimator_mce1.model_data,
    "Environment": {
        "SAGEMAKER_PROGRAM": "inference.py",
        "SAGEMAKER_SUBMIT_DIRECTORY":  estimator_mce1.model_data,
    },
}

pytorch_container2 = {
    "ContainerHostname": "pytorch-181-mnist",
    "Image": container_pt181_mce2,
    "ModelDataUrl": estimator_mce2.model_data,
    "Environment": {
        "SAGEMAKER_PROGRAM": "inference.py",
        "SAGEMAKER_SUBMIT_DIRECTORY":  estimator_mce2.model_data,
    },
}

以下のセルでは、create_model APIを呼び出して、上記で作成した PyTorch 1.8.0 と PyTorch 1.8.1 の両方のコンテナの定義を含むモデルを作成します。これは、Containers 引数の下に両方のコンテナを供給する必要があります。また、InferenceExecutionConfigフィールドのModeパラメータを、各コンテナを直接呼び出す場合はDirect、コンテナを推論パイプラインとして使用する場合はSerialに設定します。デフォルトのモードはSerialです。詳細については、[Deploy multi-container endpoints](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-container-endpoints.html) をご参照ください。

このノートブックでは Direct 呼び出しの動作に焦点を当てているため、値を `Direct` に設定します。

In [ ]:
model_name = "mnist-multi-container"
endpoint_name = model_name + '-ep'
endpoint_config_name = endpoint_name + '-config'

create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    Containers=[pytorch_container1, pytorch_container2],
    InferenceExecutionConfig={"Mode": "Direct"},
    ExecutionRoleArn=role,
)

In [ ]:
endpoint_config = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "prod",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.c5.4xlarge",
        },
    ],
)

### マルチコンテナエンドポイントの作成

作成したエンドポイント設定を使ってエンドポイントを作成します。

In [ ]:
endpoint = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

In [ ]:
describe_endpoint = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)

endpoint_status = describe_endpoint["EndpointStatus"]

while endpoint_status != "InService":
    print("Current endpoint status is: {}, Trying again...".format(endpoint_status))
    time.sleep(60)
    resp = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = resp["EndpointStatus"]

print("Endpoint status changed to 'InService'")

### 推論の実行

デプロイしたそれぞれのコンテナを指定して推論を実行します。`invoke_endpoint` の引数 `TargetContainerHostname` に使用したいコンテナ設定を指定します。

In [ ]:
payload = json.dumps(images.tolist())
result_mce1 = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    TargetContainerHostname=pytorch_container1['ContainerHostname'],
    Body=payload
)

pt_body = result_mce1["Body"].read().decode("utf-8")
predicted_label1 = np.argmax(np.array(json.loads(pt_body), dtype=np.float32), axis=1).tolist()

result_mce2 = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    TargetContainerHostname=pytorch_container2['ContainerHostname'],
    Body=payload
)

pt_body = result_mce2["Body"].read().decode("utf-8")
predicted_label2 = np.argmax(np.array(json.loads(pt_body), dtype=np.float32), axis=1).tolist()

print('The GT labels are: {}'.format(raw_labels))
print('The predicted labels (PyTorch 1.8.0) are: {}'.format(predicted_label1))
print('The predicted labels (PyTorch 1.8.1) are: {}'.format(predicted_label2))

In [ ]:

response = sagemaker_client.delete_endpoint(
    EndpointName=endpoint_name
)
response = sagemaker_client.delete_endpoint_config(
    EndpointConfigName=endpoint_config_name
)
response = sagemaker_client.delete_model(
    ModelName=model_name
)

## 非同期推論

[2021年8月に、非同期推論のための仕組みが追加されました。](https://aws.amazon.com/jp/about-aws/whats-new/2021/08/amazon-sagemaker-asynchronous-new-inference-option/)非同期推論は、推論リクエストに含めるペイロードサイズが大きかったり（最大1GBまで）、推論に時間が必要（最大15分）な場合に有効な方法です。また、非同期推論においてはオートスケーリングの設定で、最少インスタンス数をゼロに設定できます。そのため、2、3 分のコールドスタートペナルティを許容できるユースケースでは、コスト削減を図ることも可能です。

In [ ]:
bucket_prefix_async = "async-inference-demo"
resource_name_async = "AsyncInferenceDemo"
data_dir_async = "data-async"

### デプロイするモデルの学習

まず、非同期推論エンドポイントにデプロイするためのモデルを作成します。学習ジョブが作成する model.tar.gz には、学習済みモデルだけではなく、code フォルダ以下に格納された推論用コードも必要です。今回は、`code_async/train.py` の `save_model()` 関数で学習済みモデルを保存する際に、同じフォルダにソースコード一式も含めるようにしました。これにより、S3 にアップロードされる model.tar.gz の中に code フォルダと必要なスクリプトファイルが保存されます。

In [ ]:
estimator_async = PyTorch(entry_point="train.py",
                    source_dir='code_async',
                    role=role,
                    framework_version='1.8.0',
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.c5.xlarge',
                    hyperparameters={
                        'batch-size':128,
                        'lr': 0.01,
                        'epochs': 1,
                        'backend': 'gloo'
                    })
estimator_async.fit({'training': inputs})

### コンテナイメージの取得

次にプライマリコンテナを取得します。プライマリコンテナとは、推論コード、アーティファクト（学習済みモデルなど）、推論コードが予測のためにモデルをデプロイする際に使用するカスタム環境マップを含む Dockerイメージです。この例では、SageMaker が用意した PyTorch のコンテナイメージを取得します。

In [ ]:
from sagemaker import image_uris

# Specify an AWS container image and region as desired
container = image_uris.retrieve(
    region=region, framework="pytorch", 
    version="1.8.0", instance_type='ml.c5.xlarge', image_scope='inference', py_version='py36')
container

### モデルの作成

ModelName、ExecutionRoleARN （Amazon SageMaker がモデルのアーティファクトやデプロイ用の docker イメージにアクセスする際に想定される IAMロールの ARN）、PrimaryContainer を指定してモデルを作成します。

In [ ]:
model_name = resource_name_async.format("Model")
try:
    sagemaker_client.delete_model(ModelName=model_name)
except:
    pass
    
create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": container,
        "ModelDataUrl": estimator_async.model_data,
    },
)

print(f"Created Model: {create_model_response['ModelArn']}")

### エンドポイント設定の作成

モデルができたら、CreateEndpointConfig でエンドポイント設定を作成します。Amazon SageMaker ホスティングサービスは、この設定を使用してモデルをデプロイします。エンドポイント設定では、CreateModel APIを使用して作成された1つまたは複数のモデルを特定し、Amazon SageMaker にプロビジョニングさせたいリソースをデプロイします。AsyncInferenceConfig オブジェクトを指定し、OutputConfig には出力先の Amazon S3 の場所を指定します。オプションで、予測結果に関する通知を送信する Amazon SNS トピックを指定できます。

In [ ]:
endpoint_config_name = resource_name_async.format("EndpointConfig")
try:
    sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
except:
    pass

create_endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.m5.xlarge",
            "InitialInstanceCount": 1,
        }
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            "S3OutputPath": f"s3://{bucket_name}/{bucket_prefix_async}/output",
            # Optionally specify Amazon SNS topics
            # "NotificationConfig": {
            #   "SuccessTopic": "arn:aws:sns:us-east-2:123456789012:MyTopic",
            #   "ErrorTopic": "arn:aws:sns:us-east-2:123456789012:MyTopic",
            # }
        },
        "ClientConfig": {"MaxConcurrentInvocationsPerInstance": 4},
    },
)
print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

### エンドポイントの作成

モデルとエンドポイントの設定ができたら、CreateEndpoint API を使ってエンドポイントを作成します。エンドポイント名は、AWS アカウントの AWSリージョン内で一意である必要があります。

In [ ]:
endpoint_name = str(resource_name_async).format("Endpoint")
try:
    sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
except:
    pass

create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

以下のセルを実行して、エンドポイントの起動状況を確認します。ステータスが InService になっていれば推論を実行することができます。

In [ ]:
waiter = sagemaker_client.get_waiter("endpoint_in_service")
print("Waiting for endpoint to create...")
waiter.wait(EndpointName=endpoint_name)
resp = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {resp['EndpointStatus']}")

### [オプション] オートスケールポリシーの設定

このセクションでは、Application Autoscaling を使用して非同期エンドポイントにオートスケーリングを設定する方法について説明します。まず、エンドポイントのバリアントを Application Autoscaling に登録し、スケーリングポリシーを定義してから、スケーリングポリシーを適用する必要があります。この構成では、 ApproximateBacklogSizePerInstance というカスタムメトリック（CustomizedMetricSpecification）を使用します。非同期推論エンドポイントで利用可能なメトリクスの詳細なリストについては、[SageMaker 開発者ガイド](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference.html) をご参照ください。

In [ ]:
client = boto3.client(
    "application-autoscaling"
)  # Common class representing Application Auto Scaling for SageMaker amongst other services

resource_id = (
    "endpoint/" + endpoint_name + "/variant/" + "variant1"
)  # This is the format in which application autoscaling references the endpoint

# Configure Autoscaling on asynchronous endpoint down to zero instances
response = client.register_scalable_target(
    ServiceNamespace="sagemaker",
    ResourceId=resource_id,
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    MinCapacity=0,
    MaxCapacity=5,
)

response = client.put_scaling_policy(
    PolicyName="Invocations-ScalingPolicy",
    ServiceNamespace="sagemaker",  # The namespace of the AWS service that provides the resource.
    ResourceId=resource_id,  # Endpoint name
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",  # SageMaker supports only Instance Count
    PolicyType="TargetTrackingScaling",  # 'StepScaling'|'TargetTrackingScaling'
    TargetTrackingScalingPolicyConfiguration={
        "TargetValue": 5.0,  # The target value for the metric. - here the metric is - SageMakerVariantInvocationsPerInstance
        "CustomizedMetricSpecification": {
            "MetricName": "ApproximateBacklogSizePerInstance",
            "Namespace": "AWS/SageMaker",
            "Dimensions": [{"Name": "EndpointName", "Value": endpoint_name}],
            "Statistic": "Average",
        },
        "ScaleInCooldown": 600,  # The cooldown period helps you prevent your Auto Scaling group from launching or terminating
        # additional instances before the effects of previous activities are visible.
        # You can configure the length of time based on your instance startup time or other application needs.
        # ScaleInCooldown - The amount of time, in seconds, after a scale in activity completes before another scale in activity can start.
        "ScaleOutCooldown": 300  # ScaleOutCooldown - The amount of time, in seconds, after a scale out activity completes before another scale out activity can start.
        # 'DisableScaleIn': True|False - ndicates whether scale in by the target tracking policy is disabled.
        # If the value is true , scale in is disabled and the target tracking policy won't remove capacity from the scalable resource.
    },
)
response

### 非同期推論エンドポイントで推論

非同期推論を実行するには、推論の入力データを Amazon S3 にアップロードしておく必要があります。

In [ ]:
if not os.path.exists(data_dir_async):
    os.mkdir(data_dir_async)
image_list = []
for i in indices:
    test_data_loader = DataLoader([test_data[i][0]], batch_size=len(test_data))
    test_data_loaded = next(iter(test_data_loader))
    filename = os.path.join(data_dir_async, 'test_'+str(i)+'_'+str(test_data[i][1])+'.pt')
    torch.save(test_data_loaded, filename)
    image_list.append(filename)

In [ ]:
image_list

In [ ]:
inputs_async = sagemaker_session.upload_data(path=data_dir_async, bucket=bucket_name, key_prefix=os.path.join(bucket_prefix_async, 'data'))
print('input spec (in this case, just an S3 path): {}'.format(inputs_async))

**非同期推論の実行**

`invoke_endpoint_async() `を使って、非同期エンドポイントにホストされているモデルを使って推論を実行します。InputLocation フィールドに推論データの場所を、EndpointName にエンドポイントの名前を指定します。response には、結果が保存されるAmazon S3 のパスが含まれます。

このサンプルノートブックでは、5枚の画像を連続して推論します。

In [ ]:
output_locations = []
for i in image_list:
    basename = os.path.basename(i)
    print(basename)
    response = sagemaker_runtime.invoke_endpoint_async(
        EndpointName=endpoint_name, InputLocation=os.path.join(inputs_async, basename)
    )
    output_locations.append([basename, response["OutputLocation"]])
    print(f"OutputLocation: {output_locations[-1]}")

**推論結果の確認**

推論が処理されたかどうかをファイルの出力状況で確認します。推論リクエストの結果が S3 に保存されるまで、2秒ごとにファイルの有無を確認する関数 `get_output()` を定義します。

In [ ]:
import urllib, time
from botocore.exceptions import ClientError


def get_output(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    while True:
        try:
            return sagemaker_session.read_s3_file(bucket=output_url.netloc, key_prefix=output_url.path[1:])
        except ClientError as e:
            if e.response["Error"]["Code"] == "NoSuchKey":
                print("waiting for output...")
                time.sleep(2)
                continue
            raise

以下のセルを実行して、推論結果が保存されていれば、その内容を表示します。

In [ ]:
import ast, re
for f, output_location in output_locations:
    gt = int(re.split('[_\.]', f)[2])
    output = get_output(output_location)
    result = ast.literal_eval(output)[0]
    pred = np.argmax(result)
    print(f"Ground Truth: {gt}, predict: {pred}, {gt==pred}")

### リソースの削除

非同期エンドポイントと、（設定を作成している場合）オートスケール設定を削除します。

In [ ]:
# オートスケール設定の削除
response = client.describe_scaling_policies(
    ResourceId=resource_id,
    ServiceNamespace='sagemaker'
)

if len(response['ScalingPolicies']) > 0:
    response = client.deregister_scalable_target(
        ServiceNamespace='sagemaker',
        ResourceId=resource_id,
        ScalableDimension='sagemaker:variant:DesiredInstanceCount'
    )

In [ ]:
# エンドポイントの削除
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

## カスタムコンテナイメージで学習・推論

自分で作成した PyTorch コンテナを使って推論を行う場合の手順をご紹介します。ビルドしたコンテナイメージを Amazon ECR に push して Amazon SageMaker から呼び出して推論を行います。そのために、ノートブックインスタンスにアタッチされている IAM role に以下の手順でロールを追加してください。なお、`pip install` でライブラリを追加するだけであれば、[スクリプトファイル（train.py など）と同じフォルダに requirements.txt を入れておく](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html?highlight=requirements.txt#using-third-party-libraries) ことで、学習用インスタンスでコンテナが起動する際に requirements.txt に記載されたライブラリが自動的にインストールされるので、そちらをご利用いただくのも一案です。

1. Amazon SageMaker コンソールからこのノートブックインスタンスの詳細画面を表示<br>
（左側のメニューのインスタンス -> ノートブックインスタンス -> インスタンス名をクリック）
1. 　「アクセス許可と暗号化」の「IAM ロール ARN」のリンクをクリック（IAM のコンソールに遷移します）
1. 　「ポリシーをアタッチします」と書いてある青いボタンをクリック
1. 　検索ボックスに ec2containerregistry と入力し AmazonEC2ContainerRegistryFullAccess のチェックボックスをチェックする
1. 　「ポリシーのアタッチ」と書いてある青いボタンをクリック

**Amazon SageMaker Studio では、docker コマンドを使用することができません。代わりのコマンド `sm-docker` がありますので、[こちらの手順](https://sagemaker-immersionday.workshop.aws/ja/lab3/option2.html) を参考にご利用ください。**

カスタムコンテナを作成する場合、ベースイメージとして SageMaker が用意したコンテナイメージを使うと便利です。[SageMaker のコンテナイメージのリストはこちら](https://github.com/aws/deep-learning-containers/blob/master/available_images.md) でご確認いただけます。

また、以下のコマンドでコンテナイメージの URI を取得することができます。

In [ ]:
from sagemaker import image_uris

# Specify an AWS container image and region as desired
container = image_uris.retrieve(
    region=region, framework="pytorch", 
    version="1.8.0", instance_type='ml.c5.xlarge', image_scope='training', py_version='py36')
container

コンテナイメージをビルドする際に、no space left というエラーが出てビルドが失敗することがあります。ノートブックインスタンスのストレージ（EBS ボリュームサイズ）を30GBくらいに設定した上で、以下のセルを実行して docker 関連のファイル保存場所を変更することをおすすめします。

In [ ]:
!sudo /etc/init.d/docker stop
!sudo mv /var/lib/docker /home/ec2-user/SageMaker/docker
!sudo ln -s /home/ec2-user/SageMaker/docker /var/lib/docker
!sudo /etc/init.d/docker start

### 学習用コンテナイメージの作成

SageMaker の PyTorch コンテナイメージをベースに、カスタムコンテナイメージを作成します。SageMaker のビルトインイメージ以外をベースにする場合は、[こちらの手順](https://docs.aws.amazon.com/sagemaker/latest/dg/adapt-training-container.html#:~:text=Step%202%3A%20Create%20and%20upload%20the%20Dockerfile%20and%20Python%20training%20scripts) の通り Dockerfile に`RUN pip3 install sagemaker-training` を 記載するのを忘れないようにしてください。

In [ ]:
!mkdir -p docker/train
!mkdir -p docker/inference

以下のセルの FROM で始まる行の `ap-northeast-1` の部分は、このノートブックを実行しているノートブッインスタンスと同じリージョンにしてください。

In [ ]:
%%writefile docker/train/Dockerfile

FROM 763104351884.dkr.ecr.ap-northeast-1.amazonaws.com/pytorch-training:1.8.0-cpu-py36
    
RUN pip3 install  numpy

WORKDIR /

In [ ]:
ecr_repository_train = 'pytorch-byo-train'
uri_suffix = 'amazonaws.com'
tag = ':latest'
train_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository_train + tag)

# Create ECR repository and push docker image
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 763104351884.dkr.ecr.{region}.amazonaws.com
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
!docker build -t $ecr_repository_train docker/train
!aws ecr create-repository --repository-name $ecr_repository_train
!docker tag {ecr_repository_train + tag} $train_repository_uri
!docker push $train_repository_uri

作成した学習用コンテナを使ってモデルを学習します。

In [ ]:
estimator = PyTorch(entry_point="train.py",
                    image_uri=train_repository_uri,
                    source_dir='code_byoc',
                    role=role,
                    instance_count=1,
                    instance_type='ml.c5.xlarge',
#                     instance_type='local',
                    hyperparameters={
                        'batch-size':128,
                        'lr': 0.01,
                        'epochs': 10,
                        'backend': 'gloo'
                    })
estimator.fit({'training': inputs})

### 推論用コンテナイメージの作成

このサンプルでは、[こちらの Dockerfile](https://github.com/aws/deep-learning-containers/blob/master/pytorch/inference/docker/1.9/py3/Dockerfile.cpu) を参考にして独自のコンテナイメージを作成します。Dockerfile の他に、必要なファイルをダウンロードします。

In [ ]:
!wget https://raw.githubusercontent.com/aws/deep-learning-containers/master/src/deep_learning_container.py -O docker/inference/deep_learning_container.py
!wget https://raw.githubusercontent.com/aws/deep-learning-containers/master/pytorch/inference/docker/build_artifacts/config.properties -O docker/inference/config.properties
!wget https://raw.githubusercontent.com/aws/deep-learning-containers/master/pytorch/inference/docker/build_artifacts/torchserve-entrypoint.py -O docker/inference/torchserve-entrypoint.py

なお、コンテナイメージが用意されていない最新バージョンの PyTorch を使いたいなどでなければ、前の手順の学習用コンテナイメージ作成と同様に既存の SageMaker 推論用コンテナイメージをベースイメージにして独自のコンテナイメージを作成するのが簡単です。たとえば、以下のような Dockerfile で推論用カスタムイメージを作成することができます。このコードを利用する場合は、忘れずに `ap-northeast-1` の部分をお使いのリージョンに合わせて書き換えてください。

In [ ]:
# %%writefile docker/inference/Dockerfile

# FROM 763104351884.dkr.ecr.ap-northeast-1.amazonaws.com/pytorch-inference:1.8.0-cpu-py36
    
# RUN pip3 install  numpy

以下のセルを実行すると、Dockerfile が作成されます。

In [ ]:
%%writefile docker/inference/Dockerfile

FROM ubuntu:20.04

LABEL maintainer="Amazon AI"
LABEL dlc_major_version="1"
LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true
LABEL com.amazonaws.sagemaker.capabilities.multi-models=true

ARG PYTHON=python3
ARG PYTHON_VERSION=3.8.10
ARG OPEN_MPI_VERSION=4.0.1
ARG TS_VERSION=0.4.2
ARG PT_INFERENCE_URL=https://pytorch-ei-binaries.s3.us-west-2.amazonaws.com/r1.9.0_inference/20210730-020505/2dc3e2055764606a22a3c59ea955184e85ad6110/cpu/torch-1.9.0-cp38-cp38-manylinux1_x86_64.whl
ARG PT_TORCHVISION_URL=https://pytorch-ei-binaries.s3.us-west-2.amazonaws.com/torchvision_build/0.10.0/cpu/torchvision-0.10.0-cp38-cp38-linux_x86_64.whl
ARG PT_TORCHAUDIO_URL=https://pytorch-ei-binaries.s3.us-west-2.amazonaws.com/torchaudio_build/0.9.0/cpu/torchaudio-0.9.0-cp38-cp38-linux_x86_64.whl

ENV LANG C.UTF-8
ENV LD_LIBRARY_PATH /opt/conda/lib/:$LD_LIBRARY_PATH
ENV PATH /opt/conda/bin:$PATH
ENV SAGEMAKER_SERVING_MODULE sagemaker_pytorch_serving_container.serving:main
ENV TEMP=/home/model-server/tmp
# Set MKL_THREADING_LAYER=GNU to prevent issues between torch and numpy/mkl
ENV MKL_THREADING_LAYER=GNU

RUN apt-get update \
# TODO: Remove systemd upgrade once it is updated in base image
 && apt-get -y upgrade --only-upgrade systemd \
 && apt-get install -y --no-install-recommends software-properties-common \
 && add-apt-repository ppa:openjdk-r/ppa \
 && apt-get update \
 && apt-get install -y --no-install-recommends \
    build-essential \
    ca-certificates \
    cmake \
    curl \
    emacs \
    git \
    jq \
    libgl1-mesa-glx \
    libglib2.0-0 \
    libsm6 \
    libxext6 \
    libxrender-dev \
    openjdk-11-jdk \
    openssl \
    vim \
    wget \
    unzip \
    zlib1g-dev \
 && rm -rf /var/lib/apt/lists/* \
 && apt-get clean

# https://github.com/docker-library/openjdk/issues/261 https://github.com/docker-library/openjdk/pull/263/files
RUN keytool -importkeystore -srckeystore /etc/ssl/certs/java/cacerts -destkeystore /etc/ssl/certs/java/cacerts.jks -deststoretype JKS -srcstorepass changeit -deststorepass changeit -noprompt; \
    mv /etc/ssl/certs/java/cacerts.jks /etc/ssl/certs/java/cacerts; \
    /var/lib/dpkg/info/ca-certificates-java.postinst configure;

RUN curl -L -o ~/miniconda.sh https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh \
 && chmod +x ~/miniconda.sh \
 && ~/miniconda.sh -b -p /opt/conda \
 && rm ~/miniconda.sh \
 && /opt/conda/bin/conda update conda \
 && /opt/conda/bin/conda install -c conda-forge \
    python=$PYTHON_VERSION \
 && /opt/conda/bin/conda install -y \
    # conda 4.10.0 requires ruamel_yaml to be installed. Currently pinned at latest.
    ruamel_yaml==0.15.100 \
    cython \
    ipython \
    mkl-include \
    mkl \
    parso \
    scipy \
    typing \
 && /opt/conda/bin/conda clean -ya

# Conda installs links for libtinfo.so.6 and libtinfo.so.6.2 both
# Which causes "/opt/conda/lib/libtinfo.so.6: no version information available" warning
# Removing link for libtinfo.so.6. This change is needed only for ubuntu 20.04-conda, and can be reverted
# once conda fixes the issue: https://github.com/conda/conda/issues/9680
RUN rm -rf /opt/conda/lib/libtinfo.so.6

RUN wget https://www.open-mpi.org/software/ompi/v4.0/downloads/openmpi-$OPEN_MPI_VERSION.tar.gz \
 && gunzip -c openmpi-$OPEN_MPI_VERSION.tar.gz | tar xf - \
 && cd openmpi-$OPEN_MPI_VERSION \
 && ./configure --prefix=/home/.openmpi \
 && make all install \
 && cd .. \
 && rm openmpi-$OPEN_MPI_VERSION.tar.gz \
 && rm -rf openmpi-$OPEN_MPI_VERSION

# The ENV variables declared below are changed in the previous section
# Grouping these ENV variables in the first section causes
# ompi_info to fail. This is only observed in CPU containers
ENV PATH="$PATH:/home/.openmpi/bin"
ENV LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/home/.openmpi/lib/"
RUN ompi_info --parsable --all | grep mpi_built_with_cuda_support:value

RUN conda install -c \
    conda-forge \
    opencv \
 && conda install -y \
    scikit-learn \
    pandas \
    h5py \
    requests \
 && conda clean -ya \
 && pip install --upgrade pip --trusted-host pypi.org --trusted-host files.pythonhosted.org \
 && ln -s /opt/conda/bin/pip /usr/local/bin/pip3 \
 && pip install packaging==20.4 \
    enum-compat==0.0.3 \
    numpy==1.20.3 \
    "cryptography>=3.3.2"

# Uninstall and re-install torch and torchvision from the PyTorch website
RUN pip uninstall -y torch \
 && pip install --no-cache-dir -U $PT_INFERENCE_URL \
 && pip uninstall -y torchvision \
 && pip install --no-deps --no-cache-dir -U $PT_TORCHVISION_URL \
 && pip uninstall -y torchaudio \
 && pip install --no-deps --no-cache-dir -U $PT_TORCHAUDIO_URL


RUN conda install -y -c conda-forge "pyyaml>=5.4,<5.5"
RUN pip install pillow==8.3.1 "awscli<2"
RUN pip install --no-cache-dir "sagemaker-pytorch-inference>=2"

RUN pip uninstall -y model-archiver multi-model-server \
 && pip install captum \
 && pip install torchserve==$TS_VERSION \
 && pip install torch-model-archiver==$TS_VERSION

RUN cd tmp/ \
 && rm -rf tmp*

RUN useradd -m model-server \
 && mkdir -p /home/model-server/tmp /opt/ml/model \
 && chown -R model-server /home/model-server /opt/ml/model

COPY torchserve-entrypoint.py /usr/local/bin/dockerd-entrypoint.py
COPY config.properties /home/model-server

RUN chmod +x /usr/local/bin/dockerd-entrypoint.py

COPY deep_learning_container.py /usr/local/bin/deep_learning_container.py

RUN chmod +x /usr/local/bin/deep_learning_container.py

RUN HOME_DIR=/root \
 && curl -o ${HOME_DIR}/oss_compliance.zip https://aws-dlinfra-utilities.s3.amazonaws.com/oss_compliance.zip \
 && unzip ${HOME_DIR}/oss_compliance.zip -d ${HOME_DIR}/ \
 && cp ${HOME_DIR}/oss_compliance/test/testOSSCompliance /usr/local/bin/testOSSCompliance \
 && chmod +x /usr/local/bin/testOSSCompliance \
 && chmod +x ${HOME_DIR}/oss_compliance/generate_oss_compliance.sh \
 && ${HOME_DIR}/oss_compliance/generate_oss_compliance.sh ${HOME_DIR} ${PYTHON} \
 && rm -rf ${HOME_DIR}/oss_compliance*

RUN curl -o /license.txt  https://aws-dlc-licenses.s3.amazonaws.com/pytorch-1.9/license.txt

EXPOSE 8080 8081
ENTRYPOINT ["python", "/usr/local/bin/dockerd-entrypoint.py"]
CMD ["torchserve", "--start", "--ts-config", "/home/model-server/config.properties", "--model-store", "/home/model-server/"]

以下のコマンドを実行して、コンテナイメージのビルドと Amazon ECR への push を行います。ml.t2.medium のノートブックインスタンスを使用するとこのセルの実行完了までに数十分かかることがあります。念のため、セルの実行が完了したら Amazon ECR のコンソールで、ビルドしたコンテナイメージが push されているか確認しましょう。

In [ ]:
ecr_repository_inference = 'pytorch-byo-inference'
uri_suffix = 'amazonaws.com'
tag = ':latest'
inference_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository_inference + tag)

# Create ECR repository and push docker image
!docker build -t $ecr_repository_inference docker/inference
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
!aws ecr create-repository --repository-name $ecr_repository_inference
!docker tag {ecr_repository_inference + tag} $inference_repository_uri
!docker push $inference_repository_uri

### カスタムコンテナイメージを使って推論

作成した推論用コンテナイメージを使って推論してみましょう。`PyTorchModel` クラスを使って PyTorch モデルを作成します。推論エンドポイントで使用するコンテナイメージ、PyTorch バージョン、学習済みモデルのパス、推論で使用するスクリプトファイル名などを引数で指定します。

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(role=role, model_data=estimator.model_data, framework_version='1.9.0', image_uri=inference_repository_uri, source_dir="code_byoc", entry_point="inference.py")
predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

すでに起動しているエンドポイントを使って推論する場合は endpoint_name にエンドポイント名を入力して、以下のように predictor を作成します。

In [ ]:
# from sagemaker.pytorch.model import PyTorchPredictor

# predictor = PyTorchPredictor(
#         endpoint_name='<endpoint name>'
# )

それでは、カスタムコンテナイメージを使った推論エンドポイントで推論を実行します。

In [ ]:
prediction = predictor.predict(images)
predicted_label = prediction.argmax(axis=1)

print('The GT labels are: {}'.format(raw_labels))
print('The predicted labels are: {}'.format(predicted_label))

使い終わったエンドポイントを削除します。

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)

## Inf1 インスタンスで推論

Inf1インスタンスは、機械学習推論アプリケーションをサポートするためにゼロから構築されており、AWSが設計・開発した高性能機械学習推論チップである [AWS Inferentia チップ](https://aws.amazon.com/jp/machine-learning/inferentia/) を最大 16個搭載しています。このノートブックでは、学習した PyTorch モデルを Inf1インスタンスにデプロイする方法をご紹介します。Inf1 インスタンスで機械学習モデルを使用するには、モデルをコンパイルする必要があります。このノートブックでは、Amazon SageMaker Neo を使ってモデルをコンパイルします。

### モデルの学習

まずは、Inf1 で推論するためのモデルを学習します。PyTorch の場合、SageMaker Neo でモデルを変換する前にモデルに Input 情報を追加する必要があります。今回は、学習スクリプトの最後に Neo 用の変換をしておきます。詳細は、`code_inf1/train.py` の `transform_model_for_neo()` 関数をご参照ください。

In [ ]:
input_shape = [1, 1, 28, 28]

estimator_inf1 = PyTorch(entry_point="train.py",
                    source_dir='code_inf1',
                    role=role,
                    framework_version='1.8.0',
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.c5.xlarge',
#                     instance_type='local',
                    hyperparameters={
                        'batch-size':128,
                        'lr': 0.01,
                        'epochs': 1,
                        'input-shape': str(input_shape)[1:-1],
                        'backend': 'gloo'
                    })
estimator_inf1.fit({'training': inputs})

### モデルのコンパイル

モデルの学習が完了したら、SageMaker Neo でモデルをコンパイルします。まずは推論用スクリプトを指定した `PyTorchModel` を作成し、それを使って `compile` を実行します。コンパイルの際は `code_inf1/train,py` で指定した `input_size` をパラメタとして渡します。

In [ ]:
from sagemaker.pytorch.model import PyTorchModel
from datetime import datetime
from dateutil import tz
JST = tz.gettz('Asia/Tokyo')

pytorch_model = PyTorchModel(
    entry_point='inference.py',
    source_dir='code_inf1',
    model_data=estimator_inf1.model_data,
    framework_version='1.8.0',
    py_version='py3',
    role=role,
)

timestamp = datetime.now(tz=JST).strftime('%Y%m%d-%H%M%S')
compilation_job_name = "pytorch-mnist-inf1-" + timestamp
compiled_model_path = "s3://{}/{}/output".format(bucket_name, compilation_job_name)

neo_model = pytorch_model.compile(
    target_instance_family="ml_inf1",
    input_shape={"input0": input_shape},
    output_path=compiled_model_path,
    framework="pytorch",
    framework_version="1.8.0",
    role=role,
    job_name=compilation_job_name,
)

### Inf1 インスタンスで推論エンドポイントを作成し推論

In [ ]:
predictor_inf1 = neo_model.deploy(instance_type="ml.inf1.xlarge", initial_instance_count=1)

In [ ]:
results = []
for i in images:
    prediction = predictor_inf1.predict([i])
    predicted_label = prediction.argmax(axis=1)
    results.append(predicted_label[0])

print('The GT labels are: {}'.format(raw_labels))
print('The predicted labels are: {}'.format(results))

## サーバレス推論 (preview)

Amazon SageMaker で学習したモデルをサーバレスエンドポイントにデプロイして推論することができます。この機能は 2022年1月現在プレビュー中のため、本番環境での利用は非推奨です。

### デプロイするモデルの学習

サーバレスエンドポイントにデプロイするモデルを学習します。

In [ ]:
estimator_lambda = PyTorch(entry_point="train.py",
                    source_dir='code_byoc',
                    role=role,
                    framework_version='1.8.1',
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.c5.xlarge',
                    hyperparameters={
                        'batch-size':128,
                        'lr': 0.01,
                        'epochs': 1,
                        'backend': 'gloo'
                    })
estimator_lambda.fit({'training': inputs})

### サーバレスエンドポイントの作成

以下のセルを実行して、サーバレスエンドポイントを作成します。エンドポイントの起動が完了するまでに数分程度かかります。2022年1月現在、この操作は Amazon SageMaker Python SDK からではなく、AWS SDK for Python (boto3) からのみ可能です。

In [ ]:
pytorch_inference_container = sagemaker.image_uris.retrieve(
    region=region, framework="pytorch", 
    version="1.8.1", instance_type='ml.c5.xlarge', image_scope='inference', py_version='py36')
    
model_name = f"DEMO-pytorch-serverless-{datetime.now():%Y-%m-%d-%H-%M-%S}"

endpoint_config_name = model_name + '-EndpointConfig'
endpoint_name = model_name + '-Endpoint'
response = sagemaker_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': pytorch_inference_container,
        'ModelDataUrl': estimator_lambda.model_data,
    },
    ExecutionRoleArn=role,
)
response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTrafic',
            'ModelName': model_name,
            'ServerlessConfig': { # 通常のリアルタイム推論とは違い、ServerlessConfig というキーで設定する
                'MemorySizeInMB': 1024, # メモリサイズは 1024 , 2048, 3072, 4096, 5120, 6144 から選ぶ
                'MaxConcurrency': 3 # 最大同時起動数
            }
        },
    ],
)
response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

describe_endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

### サーバレス推論の実行

以下のセルを実行して推論してみましょう。初回の実行時は推論結果が返ってくるまでに数分かかることがありますが、続けて実行するとすぐに結果が返ってくるようになります。

In [ ]:
import io
import json
buffer= io.BytesIO()
np.save(buffer, images)

response = sagemaker_runtime.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType="application/x-npy",
                                   Body=buffer.getvalue())

body = response['Body']
prediction = np.array(json.load(body))
predicted_label = prediction.argmax(axis=1)

print('The GT labels are: {}'.format(raw_labels))
print('The predicted labels are: {}'.format(predicted_label))

使い終わったエンドポイントを削除します。

In [ ]:
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

## リソースの削除

エンドポイントは、[SageMaker pricing page](https://aws.amazon.com/sagemaker/pricing/) に記載の通り、稼働している時間に応じて課金されるため、使用しなくなったら削除する必要があります。整理整頓のために、不要なエンドポイント設定も削除することをおすすめします。SageMaker のコンソールのエンドポイント一覧画面で、エンドポイントが削除されているかどうかを確認できます（一覧に表示されていなければ削除されたということです）。

また、このノートブックを実行したノートブックインスタンスも削除が必要です。インスタンスを「停止」しただけでは EBS ボリュームへの課金は継続するので、完全に課金を止めるためにインスタンスを「停止」してから「削除」を実施してください。なお、削除したあとはインスタンスに保存されているファイルなどにアクセスすることはできません。